# COMP7305 Assignment4 

Basic Infomation:

Dataset selected: Bank Marketing Dataset

Kaggle link: https://www.kaggle.com/datasets/janiobachmann/bank-marketing-dataset/data

Source: [Moro et al., 2014] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014

Hint: This ipynb file is for local envrionment testing only.

In [1]:
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, format_number, count, when, floor, desc
from pyspark.sql.window import Window

In [2]:
spark = (SparkSession
  .builder
  .appName("bank-eda")
  .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/03 11:10:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
bank_file = "bank.csv"

bank_df = (spark.read.format("csv") 
  .option("header", "true") 
  .option("inferSchema", "true") 
  .load(bank_file))

In [4]:
print(f"Number of rows: {bank_df.count()}")
print(f"Number of columns: {len(bank_df.columns)}")

Number of rows: 11162
Number of columns: 17


## 1. Expolre whether the marriage will lead to debt

In [5]:
loan_rate_by_marital = bank_df.groupBy("marital").agg(
    format_number(avg("balance"), 2).alias("avg_balance"),
    (count(when(col("loan") == "yes", True)) / count("*")).alias("loan_yes_rate")
).orderBy(col("loan_yes_rate").desc())

loan_rate_by_marital.show()

overall_loan_rate = round(
    bank_df.where(col("loan") == "yes").count() / bank_df.count(), 2
)
print(f"Overall Loan Rate: {overall_loan_rate:.2%}")

+--------+-----------+-------------------+
| marital|avg_balance|      loan_yes_rate|
+--------+-----------+-------------------+
|divorced|   1,371.84| 0.1554524361948956|
| married|   1,599.93| 0.1437568886789482|
|  single|   1,457.26|0.09835133598635588|
+--------+-----------+-------------------+

Overall Loan Rate: 13.00%


In [6]:
loan_rate_by_marital.write.csv("loan_rate_by_marital", mode="overwrite", header=True)
loan_rate_by_marital.explain("extended")

== Parsed Logical Plan ==
'Sort ['loan_yes_rate DESC NULLS LAST], true
+- Aggregate [marital#19], [marital#19, format_number(avg(balance#22), 2) AS avg_balance#91, (cast(count(CASE WHEN (loan#24 = yes) THEN true END) as double) / cast(count(1) as double)) AS loan_yes_rate#94]
   +- Relation [age#17,job#18,marital#19,education#20,default#21,balance#22,housing#23,loan#24,contact#25,day#26,month#27,duration#28,campaign#29,pdays#30,previous#31,poutcome#32,deposit#33] csv

== Analyzed Logical Plan ==
marital: string, avg_balance: string, loan_yes_rate: double
Sort [loan_yes_rate#94 DESC NULLS LAST], true
+- Aggregate [marital#19], [marital#19, format_number(avg(balance#22), 2) AS avg_balance#91, (cast(count(CASE WHEN (loan#24 = yes) THEN true END) as double) / cast(count(1) as double)) AS loan_yes_rate#94]
   +- Relation [age#17,job#18,marital#19,education#20,default#21,balance#22,housing#23,loan#24,contact#25,day#26,month#27,duration#28,campaign#29,pdays#30,previous#31,poutcome#32,deposit#

## 2. Check the loan rate and housing rate according to different kinds of jobs. And find those with higher housing rate.

In [7]:
loan_rate_by_job = bank_df.groupBy("job").agg(
    format_number(avg("balance"), 2).alias("avg_balance"),
    (count(when(col("loan") == "yes", True)) / count("*")).alias("loan_yes_rate"),
    (count(when(col("housing") == "yes", True)) / count("*")).alias("housing_yes_rate")
).orderBy(col("housing_yes_rate").desc())

loan_rate_by_job.show()

overall_housing_rate = round(
    bank_df.where(col("housing") == "yes").count() / bank_df.count(), 2
)
print(f"Overall Housing Rate: {overall_housing_rate:.2%}")

+-------------+-----------+--------------------+-------------------+
|          job|avg_balance|       loan_yes_rate|   housing_yes_rate|
+-------------+-----------+--------------------+-------------------+
|  blue-collar|   1,203.93| 0.17335390946502058| 0.6795267489711934|
|     services|   1,081.17|  0.1668472372697725| 0.6132177681473456|
|       admin.|   1,195.87| 0.17691154422788605| 0.5547226386806596|
| entrepreneur|   1,621.94| 0.21341463414634146| 0.5060975609756098|
|   technician|   1,556.29| 0.13603949533735601| 0.4805266044980801|
|self-employed|   1,865.37|  0.1382716049382716|0.42962962962962964|
|   management|   1,793.66| 0.10132501948558068| 0.4181605611847233|
|   unemployed|   1,314.72|0.058823529411764705| 0.3137254901960784|
|    housemaid|   1,366.16|   0.072992700729927| 0.2773722627737226|
|      retired|   2,417.25| 0.07069408740359898|0.15809768637532134|
|      student|   1,500.78|0.002777777777777778|0.14166666666666666|
|      unknown|   1,945.46| 0.0285

In [8]:
loan_rate_by_job.write.csv("loan_rate_by_job", mode="overwrite", header=True)
loan_rate_by_job.explain("extended")

== Parsed Logical Plan ==
'Sort ['housing_yes_rate DESC NULLS LAST], true
+- Aggregate [job#18], [job#18, format_number(avg(balance#22), 2) AS avg_balance#210, (cast(count(CASE WHEN (loan#24 = yes) THEN true END) as double) / cast(count(1) as double)) AS loan_yes_rate#213, (cast(count(CASE WHEN (housing#23 = yes) THEN true END) as double) / cast(count(1) as double)) AS housing_yes_rate#216]
   +- Relation [age#17,job#18,marital#19,education#20,default#21,balance#22,housing#23,loan#24,contact#25,day#26,month#27,duration#28,campaign#29,pdays#30,previous#31,poutcome#32,deposit#33] csv

== Analyzed Logical Plan ==
job: string, avg_balance: string, loan_yes_rate: double, housing_yes_rate: double
Sort [housing_yes_rate#216 DESC NULLS LAST], true
+- Aggregate [job#18], [job#18, format_number(avg(balance#22), 2) AS avg_balance#210, (cast(count(CASE WHEN (loan#24 = yes) THEN true END) as double) / cast(count(1) as double)) AS loan_yes_rate#213, (cast(count(CASE WHEN (housing#23 = yes) THEN true

## 3. Explore the relationship between age groups and average deposits and show the housing rates of different age groups.

In [9]:
age_balance_housing_stats = bank_df \
    .withColumn("age_group", floor(col("age") / 10) * 10) \
    .groupBy("age_group") \
    .agg(
        format_number(avg("balance"), 2).alias("avg_balance"),
        avg(when(col("housing") == "yes", 1).otherwise(0)).alias("housing_rate")
    ) \
    .orderBy(col("avg_balance").desc())


age_balance_housing_stats \
    .where(col("age_group").isin([20, 30, 40, 50, 60])) \
    .show()


+---------+-----------+-------------------+
|age_group|avg_balance|       housing_rate|
+---------+-----------+-------------------+
|       60|   2,481.71| 0.1797520661157025|
|       50|   1,798.21|  0.386737400530504|
|       40|   1,483.56| 0.5235920852359208|
|       30|   1,350.98| 0.5560444650301065|
|       20|   1,183.98|0.44313725490196076|
+---------+-----------+-------------------+



In [10]:
age_balance_housing_stats.write.csv("age_balance_housing_stats", mode="overwrite", header=True)
age_balance_housing_stats.explain("extended")

== Parsed Logical Plan ==
'Sort ['avg_balance DESC NULLS LAST], true
+- Aggregate [age_group#329L], [age_group#329L, format_number(avg(balance#22), 2) AS avg_balance#367, avg(CASE WHEN (housing#23 = yes) THEN 1 ELSE 0 END) AS housing_rate#369]
   +- Project [age#17, job#18, marital#19, education#20, default#21, balance#22, housing#23, loan#24, contact#25, day#26, month#27, duration#28, campaign#29, pdays#30, previous#31, poutcome#32, deposit#33, (FLOOR((cast(age#17 as double) / cast(10 as double))) * cast(10 as bigint)) AS age_group#329L]
      +- Relation [age#17,job#18,marital#19,education#20,default#21,balance#22,housing#23,loan#24,contact#25,day#26,month#27,duration#28,campaign#29,pdays#30,previous#31,poutcome#32,deposit#33] csv

== Analyzed Logical Plan ==
age_group: bigint, avg_balance: string, housing_rate: double
Sort [avg_balance#367 DESC NULLS LAST], true
+- Aggregate [age_group#329L], [age_group#329L, format_number(avg(balance#22), 2) AS avg_balance#367, avg(CASE WHEN (housi

## 4. Education stat including the distribution of the education situation, the avg balance and housing rate of each education option.

In [11]:
edu_stats = bank_df.where(col("education") != "unknown") \
                  .groupBy("education") \
                  .agg(count("*").alias("count"),
                       format_number(avg("balance"), 2).alias("avg_balance"),
                       (count(when(col("housing") == "yes", True)) / count("*")).alias("housing_rate")) \
                  .orderBy(desc("avg_balance"), desc("housing_rate"))

edu_stats.show()

+---------+-----+-----------+-------------------+
|education|count|avg_balance|       housing_rate|
+---------+-----+-----------+-------------------+
| tertiary| 3689|   1,845.87| 0.3914339929520195|
|  primary| 1500|   1,523.03|0.49466666666666664|
|secondary| 5476|   1,296.48|  0.533418553688824|
+---------+-----+-----------+-------------------+



In [12]:
edu_stats.write.csv("edu_stats", mode="overwrite", header=True)
edu_stats.explain("extended")

== Parsed Logical Plan ==
'Sort ['avg_balance DESC NULLS LAST, 'housing_rate DESC NULLS LAST], true
+- Aggregate [education#20], [education#20, count(1) AS count#445L, format_number(avg(balance#22), 2) AS avg_balance#447, (cast(count(CASE WHEN (housing#23 = yes) THEN true END) as double) / cast(count(1) as double)) AS housing_rate#450]
   +- Filter NOT (education#20 = unknown)
      +- Relation [age#17,job#18,marital#19,education#20,default#21,balance#22,housing#23,loan#24,contact#25,day#26,month#27,duration#28,campaign#29,pdays#30,previous#31,poutcome#32,deposit#33] csv

== Analyzed Logical Plan ==
education: string, count: bigint, avg_balance: string, housing_rate: double
Sort [avg_balance#447 DESC NULLS LAST, housing_rate#450 DESC NULLS LAST], true
+- Aggregate [education#20], [education#20, count(1) AS count#445L, format_number(avg(balance#22), 2) AS avg_balance#447, (cast(count(CASE WHEN (housing#23 = yes) THEN true END) as double) / cast(count(1) as double)) AS housing_rate#450]


## 5. Find those with higher posibility of default according to jobs

In [13]:
job_default = bank_df.groupBy(
    "job"
).agg(
    count(when(col("default") == "yes", True)).alias("default_count"),
    count(when(col("default") == "no", True)).alias("non_default_count"),
    count("*").alias("total_count")
).withColumn(
    "default_rate", col("default_count") / col("total_count")
).orderBy(desc("default_rate"))

job_default.show()

+-------------+-------------+-----------------+-----------+--------------------+
|          job|default_count|non_default_count|total_count|        default_rate|
+-------------+-------------+-----------------+-----------+--------------------+
| entrepreneur|           10|              318|        328| 0.03048780487804878|
|    housemaid|            8|              266|        274|0.029197080291970802|
|   unemployed|            8|              349|        357|0.022408963585434174|
|  blue-collar|           41|             1903|       1944| 0.02109053497942387|
|self-employed|            8|              397|        405|0.019753086419753086|
|   technician|           29|             1794|       1823|0.015907844212835986|
|   management|           39|             2527|       2566|0.015198752922837101|
|      unknown|            1|               69|         70|0.014285714285714285|
|       admin.|           11|             1323|       1334|0.008245877061469266|
|     services|            7

In [14]:
job_default.write.csv("job_default", mode="overwrite", header=True)
job_default.explain("extended")

Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


CodeCache: size=131072Kb used=34872Kb max_used=34886Kb free=96199Kb
 bounds [0x00000001069e0000, 0x0000000108c30000, 0x000000010e9e0000]
 total_blobs=12776 nmethods=11780 adapters=908
 compilation: disabled (not enough contiguous free space left)
== Parsed Logical Plan ==
'Sort ['default_rate DESC NULLS LAST], true
+- Project [job#18, default_count#526L, non_default_count#528L, total_count#530L, (cast(default_count#526L as double) / cast(total_count#530L as double)) AS default_rate#535]
   +- Aggregate [job#18], [job#18, count(CASE WHEN (default#21 = yes) THEN true END) AS default_count#526L, count(CASE WHEN (default#21 = no) THEN true END) AS non_default_count#528L, count(1) AS total_count#530L]
      +- Relation [age#17,job#18,marital#19,education#20,default#21,balance#22,housing#23,loan#24,contact#25,day#26,month#27,duration#28,campaign#29,pdays#30,previous#31,poutcome#32,deposit#33] csv

== Analyzed Logical Plan ==
job: string, default_count: bigint, non_default_count: bigint, tota